In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Data

In [3]:
train = pd.read_csv("location_train.csv")
test = pd.read_csv("location_test.csv")

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Columns: 448 entries, ID to 446
dtypes: int64(448)
memory usage: 13.7 MB


In [5]:
train.head()

,ID,class,1,2,3,4,5,6,7,8,...,437,438,439,440,441,442,443,444,445,446
0,0,11,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,8,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Columns: 447 entries, ID to 446
dtypes: int64(447)
memory usage: 3.4 MB


In [7]:
test.head()

,ID,1,2,3,4,5,6,7,8,9,...,437,438,439,440,441,442,443,444,445,446
0,4000,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,4001,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,4002,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4003,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,4004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [8]:
X_train = train.drop(["ID", "class"], axis=1)
y_train = train["class"]

X_test = test.drop(["ID"], axis=1)

# Pre-processing

In [9]:
# not necessary

# Exploratory data analysis

In [10]:
X_train.isnull().sum()

1      0
2      0
3      0
4      0
5      0
      ..
442    0
443    0
444    0
445    0
446    0
Length: 446, dtype: int64

# Model selection

In [11]:
# Grid Search

cv = 5         # number of folds
verbose = 1     # information shown during training

## Logistic Regression

In [12]:
parameters = {
    "penalty":["l1", "l2", "elasticnet", "none"], 
    "C":[0.001, 0.01 ,0.1, 1, 10],
    "class_weight":["None", "balanced"],
    "solver":["liblinear"]}
model = GridSearchCV(LogisticRegression(), parameters, cv=cv, verbose=verbose, scoring="f1_weighted")
model.fit(X_train, y_train)

results = pd.DataFrame(model.cv_results_)
results = results[["param_penalty", "param_C", "param_class_weight", "mean_test_score"]]
results.sort_values(["mean_test_score"], ascending=False).head(10)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


,param_penalty,param_C,param_class_weight,mean_test_score
13,l2,0.01,balanced,0.672487
21,l2,0.1,balanced,0.648258
5,l2,0.001,balanced,0.588957
28,l1,1,balanced,0.586562
29,l2,1,balanced,0.582844
20,l1,0.1,balanced,0.553211
37,l2,10,balanced,0.545983
36,l1,10,balanced,0.538411
45,l2,100,balanced,0.534584
44,l1,100,balanced,0.511661


# Final model

In [13]:
best_model = model.best_estimator_
best_model.fit(X_train, y_train)

LogisticRegression(C=0.01, class_weight='balanced', solver='liblinear')

In [14]:
predictions = pd.DataFrame(test["ID"])
predictions["class"] = best_model.predict(X_test)

In [15]:
predictions.to_csv("submission.csv", index=False)